####  Bayesian Optimization을 이용하여 application과 previous로 만들어진 집합의 하이퍼 파라미터 튜닝

In [1]:
import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [2]:
def get_dataset():
    app_train = pd.read_csv('application_train.csv')
    app_test = pd.read_csv('application_test.csv')
    apps = pd.concat([app_train, app_test])
    prev = pd.read_csv('previous_application.csv')

    return apps, prev

apps, prev = get_dataset()

In [3]:
def get_apps_processed(apps):
    
    # EXT_SOURCE_X FEATURE 가공
    apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())
    
    # AMT_CREDIT 비율로 Feature 가공
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    
    # AMT_INCOME_TOTAL 비율로 Feature 가공
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH, DAYS_EMPLOYED 비율로 Feature 가공
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
    apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
    apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']
    
    return apps

In [4]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

def get_prev_processed(prev):
    # 대출 신청 금액과 실제 대출액/대출 상품금액 차이 및 비율
    prev['PREV_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
    prev['PREV_GOODS_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_GOODS_PRICE']
    prev['PREV_CREDIT_APPL_RATIO'] = prev['AMT_CREDIT']/prev['AMT_APPLICATION']
    # prev['PREV_ANNUITY_APPL_RATIO'] = prev['AMT_ANNUITY']/prev['AMT_APPLICATION']
    prev['PREV_GOODS_APPL_RATIO'] = prev['AMT_GOODS_PRICE']/prev['AMT_APPLICATION']
    
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # 첫번째 만기일과 마지막 만기일까지의 기간
    prev['PREV_DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']
    # 매월 납부 금액과 납부 횟수 곱해서 전체 납부 금액 구함. 
    all_pay = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    # 전체 납부 금액 대비 AMT_CREDIT 비율을 구하고 여기에 다시 납부횟수로 나누어서 이자율 계산. 
    prev['PREV_INTERESTS_RATE'] = (all_pay/prev['AMT_CREDIT'] - 1)/prev['CNT_PAYMENT']
        
    return prev
    
    
def get_prev_amt_agg(prev):
    # 새롭게 생성된 대출 신청액 대비 다른 금액 차이 및 비율로 aggregation 수행. 
    agg_dict = {
         # 기존 컬럼. 
        'SK_ID_CURR':['count'],
        'AMT_CREDIT':['mean', 'max', 'sum'],
        'AMT_ANNUITY':['mean', 'max', 'sum'], 
        'AMT_APPLICATION':['mean', 'max', 'sum'],
        'AMT_DOWN_PAYMENT':['mean', 'max', 'sum'],
        'AMT_GOODS_PRICE':['mean', 'max', 'sum'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        # 가공 컬럼
        'PREV_CREDIT_DIFF':['mean', 'max', 'sum'], 
        'PREV_CREDIT_APPL_RATIO':['mean', 'max'],
        'PREV_GOODS_DIFF':['mean', 'max', 'sum'],
        'PREV_GOODS_APPL_RATIO':['mean', 'max'],
        'PREV_DAYS_LAST_DUE_DIFF':['mean', 'max', 'sum'],
        'PREV_INTERESTS_RATE':['mean', 'max']
    }

    prev_group = prev.groupby('SK_ID_CURR')
    prev_amt_agg = prev_group.agg(agg_dict)

    # multi index 컬럼을 '_'로 연결하여 컬럼명 변경
    prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]
    
    return prev_amt_agg

def get_prev_refused_appr_agg(prev):
    # 원래 groupby 컬럼 + 세부 기준 컬럼으로 groupby 수행. 세분화된 레벨로 aggregation 수행 한 뒤에 unstack()으로 컬럼레벨로 변형. 
    prev_refused_appr_group = prev[prev['NAME_CONTRACT_STATUS'].isin(['Approved', 'Refused'])].groupby([ 'SK_ID_CURR', 'NAME_CONTRACT_STATUS'])
    prev_refused_appr_agg = prev_refused_appr_group['SK_ID_CURR'].count().unstack()
    # 컬럼명 변경. 
    prev_refused_appr_agg.columns = ['PREV_APPROVED_COUNT', 'PREV_REFUSED_COUNT' ]
    # NaN값은 모두 0으로 변경. 
    prev_refused_appr_agg = prev_refused_appr_agg.fillna(0)
    
    return prev_refused_appr_agg

    

def get_prev_agg(prev):
    prev = get_prev_processed(prev)
    prev_amt_agg = get_prev_amt_agg(prev)
    prev_refused_appr_agg = get_prev_refused_appr_agg(prev)
    
    # prev_amt_agg와 조인. 
    prev_agg = prev_amt_agg.merge(prev_refused_appr_agg, on='SK_ID_CURR', how='left')
    # SK_ID_CURR별 과거 대출건수 대비 APPROVED_COUNT 및 REFUSED_COUNT 비율 생성. 
    prev_agg['PREV_REFUSED_RATIO'] = prev_agg['PREV_REFUSED_COUNT']/prev_agg['PREV_SK_ID_CURR_COUNT']
    prev_agg['PREV_APPROVED_RATIO'] = prev_agg['PREV_APPROVED_COUNT']/prev_agg['PREV_SK_ID_CURR_COUNT']
    # 'PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT' 컬럼 drop 
    prev_agg = prev_agg.drop(['PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT'], axis=1)
    
    return prev_agg

def get_apps_all_with_prev_agg(apps, prev):
    apps_all =  get_apps_processed(apps)
    prev_agg = get_prev_agg(prev)
    print('prev_agg shape:', prev_agg.shape)
    print('apps_all before merge shape:', apps_all.shape)
    apps_all = apps_all.merge(prev_agg, on='SK_ID_CURR', how='left')
    print('apps_all after merge with prev_agg shape:', apps_all.shape)
    
    return apps_all

def get_apps_all_encoded(apps_all):
    object_columns = apps_all.dtypes[apps_all.dtypes == 'object'].index.tolist()
    for column in object_columns:
        apps_all[column] = pd.factorize(apps_all[column])[0]
    
    return apps_all

def get_apps_all_train_test(apps_all):
    apps_all_train = apps_all[~apps_all['TARGET'].isnull()]
    apps_all_test = apps_all[apps_all['TARGET'].isnull()]

    apps_all_test = apps_all_test.drop('TARGET', axis=1)
    
    return apps_all_train, apps_all_test
    
def train_apps_all(apps_all_train):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
    print('train shape:', train_x.shape, 'valid shape:', valid_x.shape)
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=2000,
                learning_rate=0.01,
                num_leaves=32,
                colsample_bytree=0.8,
                subsample=0.8,
                max_depth=8,
                reg_alpha=0.04,
                reg_lambda=0.07,
                min_child_weight=40,
                silent=-1,
                verbose=-1,
                )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    
    return clf

In [5]:
apps_all = get_apps_all_with_prev_agg(apps, prev)
apps_all = get_apps_all_encoded(apps_all)
apps_all_train, apps_all_test = get_apps_all_train_test(apps_all)
ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = apps_all_train['TARGET']
train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)


C:\Users\miming\AppData\Local\Temp\ipykernel_4892\1480129953.py:53: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]


prev_agg shape: (338857, 41)
apps_all before merge shape: (356255, 135)
apps_all after merge with prev_agg shape: (356255, 176)


### Bayesian Optimization

In [6]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

In [7]:
# 함수의 입력값 search 범위(하이퍼 파라미터 별 입력 범위) 설정하기
baysian_params = {
    'max_depth' : (6, 16),
    'num_leaves': (24, 64),
    'min_child_samples': (10, 200),
    'min_child_weight': (1, 50),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1),
    'max_bin': (10, 500),
    'reg_lambda': (0.001, 10),
    'reg_alpha': (0.01, 50)
}

In [8]:
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight,
                 subsample, colsample_bytree, max_bin, reg_lambda, reg_alpha):
    params = {
        'n_estimators': 500, 'learning_rate': 0.02,
        'max_depth': int(round(max_depth)),
        'num_leaves': int(round(num_leaves)),
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), # 음수가 들어오거나, 1보다 커질때 제한을 해줌
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_bin': max(int(round(max_bin)), 10),
        'reg_lambda': max(reg_lambda, 0),
        'reg_alpha': max(reg_alpha, 0)
    }
    
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=100,
                  early_stopping_rounds=100)
    valid_proba = lgb_model.predict_proba(valid_x)[:, 1]
    roc_auc = roc_auc_score(valid_y, valid_proba)
    return roc_auc


In [9]:
lbgBO = BayesianOptimization(
    f=lgb_roc_eval,
    pbounds=baysian_params,
    random_state=1
)

lbgBO.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------


c:\Python310\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.762504	training's binary_logloss: 0.2479	valid_1's auc: 0.753638	valid_1's binary_logloss: 0.24943
[200]	training's auc: 0.778797	training's binary_logloss: 0.241178	valid_1's auc: 0.764735	valid_1's binary_logloss: 0.244671
[300]	training's auc: 0.788513	training's binary_logloss: 0.237491	valid_1's auc: 0.770055	valid_1's binary_logloss: 0.242744
[400]	training's auc: 0.795492	training's binary_logloss: 0.234889	valid_1's auc: 0.772857	valid_1's binary_logloss: 0.241745
[500]	training's auc: 0.801482	training's binary_logloss: 0.23268	valid_1's auc: 0.774752	valid_1's binary_logloss: 0.241084
| 1         | 0.7748    | 0.7085    | 363.0     | 6.001     | 67.44     | 8.191     | 27.69     | 9.321     | 3.456     | 0.6984    |


c:\Python310\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.758305	training's binary_logloss: 0.249178	valid_1's auc: 0.751171	valid_1's binary_logloss: 0.250295
[200]	training's auc: 0.773693	training's binary_logloss: 0.242877	valid_1's auc: 0.762591	valid_1's binary_logloss: 0.245534
[300]	training's auc: 0.782466	training's binary_logloss: 0.239579	valid_1's auc: 0.767914	valid_1's binary_logloss: 0.243582
[400]	training's auc: 0.788481	training's binary_logloss: 0.237356	valid_1's auc: 0.770862	valid_1's binary_logloss: 0.242523
[500]	training's auc: 0.793375	training's binary_logloss: 0.23563	valid_1's auc: 0.77268	valid_1's binary_logloss: 0.241896
| 2         | 0.7727    | 0.7694    | 215.4     | 12.85     | 48.85     | 44.03     | 25.1      | 33.53     | 4.174     | 0.7793    |


...

[100]	training's auc: 0.782237	training's binary_logloss: 0.241871	valid_1's auc: 0.759836	valid_1's binary_logloss: 0.247288
[200]	training's auc: 0.805036	training's binary_logloss: 0.23209	valid_1's auc: 0.769754	valid_1's binary_logloss: 0.242751
[300]	training's auc: 0.821977	training's binary_logloss: 0.225544	valid_1's auc: 0.773663	valid_1's binary_logloss: 0.241275
[400]	training's auc: 0.836157	training's binary_logloss: 0.220134	valid_1's auc: 0.775896	valid_1's binary_logloss: 0.240522
[500]	training's auc: 0.848054	training's binary_logloss: 0.215562	valid_1's auc: 0.777102	valid_1's binary_logloss: 0.240092
| 28        | 0.7771    | 0.71      | 494.0     | 10.77     | 162.3     | 31.09     | 60.92     | 0.1603    | 0.9464    | 0.5215    |


c:\Python310\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.761858	training's binary_logloss: 0.247982	valid_1's auc: 0.753428	valid_1's binary_logloss: 0.249459
[200]	training's auc: 0.778724	training's binary_logloss: 0.24109	valid_1's auc: 0.765405	valid_1's binary_logloss: 0.244501
[300]	training's auc: 0.7888	training's binary_logloss: 0.237295	valid_1's auc: 0.770746	valid_1's binary_logloss: 0.242547
[400]	training's auc: 0.796399	training's binary_logloss: 0.234516	valid_1's auc: 0.773817	valid_1's binary_logloss: 0.241459
[500]	training's auc: 0.802752	training's binary_logloss: 0.232224	valid_1's auc: 0.775655	valid_1's binary_logloss: 0.240807
| 29        | 0.7757    | 0.7145    | 406.7     | 14.3      | 133.6     | 29.27     | 24.92     | 6.614     | 4.345     | 0.5224    |


c:\Python310\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's auc: 0.776723	training's binary_logloss: 0.244378	valid_1's auc: 0.759144	valid_1's binary_logloss: 0.248089
[200]	training's auc: 0.795496	training's binary_logloss: 0.235765	valid_1's auc: 0.769046	valid_1's binary_logloss: 0.243183
[300]	training's auc: 0.809237	training's binary_logloss: 0.230399	valid_1's auc: 0.773284	valid_1's binary_logloss: 0.241536
[400]	training's auc: 0.820955	training's binary_logloss: 0.225945	valid_1's auc: 0.775525	valid_1's binary_logloss: 0.240714
[500]	training's auc: 0.831519	training's binary_logloss: 0.222	valid_1's auc: 0.777019	valid_1's binary_logloss: 0.240179
| 30        | 0.777     | 0.6166    | 498.4     | 12.9      | 170.8     | 47.75     | 52.59     | 4.512     | 8.353     | 0.9305    |


In [10]:
lbgBO.res

[{'target': 0.7747515318161347,
  'params': {'colsample_bytree': 0.7085110023512871,
   'max_bin': 362.9590017866575,
   'max_depth': 6.001143748173449,
   'min_child_samples': 67.44318880004955,
   'min_child_weight': 8.19103865003854,
   'num_leaves': 27.69354379075191,
   'reg_alpha': 9.321147966769768,
   'reg_lambda': 3.4562617097034343,
   'subsample': 0.698383737115335}},
 {'target': 0.7726800872577807,
  'params': {'colsample_bytree': 0.7694083670016785,
   'max_bin': 215.40531205761445,
   'max_depth': 12.852195003967594,
   'min_child_samples': 48.84592744898831,
   'min_child_weight': 44.02775438315633,
   'num_leaves': 25.09550372791705,
   'reg_alpha': 33.52667083381833,
   'reg_lambda': 4.173630718868903,
   'subsample': 0.7793449142228759}},
 {'target': 0.7731216247078669,
  'params': {'colsample_bytree': 0.5701934692976169,
   'max_bin': 107.0697296515906,
   'max_depth': 14.007445686755366,
   'min_child_samples': 193.96969938668553,
   'min_child_weight': 16.357784729

In [11]:
# lgbBO.res에서 target이 max인것만 추려내기
target_list = []
for result in lbgBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)

# argmax를 통해 가장 큰 target의 index가져오기
print('maximum target index:', np.argmax(np.array(target_list)))

[0.7747515318161347, 0.7726800872577807, 0.7731216247078669, 0.7742910108353326, 0.7738070427875996, 0.7731230971982839, 0.7732957185047409, 0.7761834961834931, 0.7732448310149904, 0.7750903841065592, 0.7745394471311235, 0.7734197984577427, 0.7738622429110955, 0.7759966455683464, 0.7724974571464506, 0.7734608963483132, 0.7739129381987528, 0.7746834716253075, 0.775070804908199, 0.7726797489185586, 0.7738176107258349, 0.7743523074186134, 0.7744248629145971, 0.7722034149473564, 0.7736508364929756, 0.7772857958381745, 0.7770127624397868, 0.7771019060883321, 0.7756554201380265, 0.7770188493688892]
maximum target index: 25


In [12]:
# 가장 큰 target값을 가지는 params 추출
max_dict = lbgBO.res[np.argmax(np.array(target_list))]
print(max_dict)

{'target': 0.7772857958381745, 'params': {'colsample_bytree': 0.6790874358630945, 'max_bin': 495.5883417872887, 'max_depth': 13.495577949310704, 'min_child_samples': 154.92432612082445, 'min_child_weight': 41.056153505864344, 'num_leaves': 59.614555940082454, 'reg_alpha': 10.88727137584814, 'reg_lambda': 7.263008197687969, 'subsample': 0.9833530340999077}}


In [13]:
# 최적화된 파라미터로 다시 테스트
def train_apps_all(apps_all_train):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
    print('train shape:', train_x.shape, 'valid shape:', valid_x.shape)
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=1000,
                learning_rate=0.02,
                max_depth = 13,
                num_leaves=60,
                colsample_bytree=0.679,
                subsample=0.983,
                max_bin=495,
                reg_alpha=10.887,
                reg_lambda=7.2630,
                min_child_weight=41,
                min_child_samples=154,
                silent=-1,
                verbose=-1,
                )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    
    return clf

In [14]:
apps_all = get_apps_all_with_prev_agg(apps, prev)
apps_all = get_apps_all_encoded(apps_all)
apps_all_train, apps_all_test = get_apps_all_train_test(apps_all)
clf = train_apps_all(apps_all_train)

C:\Users\miming\AppData\Local\Temp\ipykernel_4892\1480129953.py:53: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  prev_amt_agg.columns = ["PREV_"+ "_".join(x).upper() for x in prev_amt_agg.columns.ravel()]


prev_agg shape: (338857, 41)
apps_all before merge shape: (356255, 135)
apps_all after merge with prev_agg shape: (356255, 176)
train shape: (215257, 174) valid shape: (92254, 174)


c:\Python310\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Python310\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.777229	training's binary_logloss: 0.244199	valid_1's auc: 0.758265	valid_1's binary_logloss: 0.248217
[200]	training's auc: 0.796064	training's binary_logloss: 0.235625	valid_1's auc: 0.768193	valid_1's binary_logloss: 0.243475
[300]	training's auc: 0.809745	training's binary_logloss: 0.230233	valid_1's auc: 0.7729	valid_1's binary_logloss: 0.241713
[400]	training's auc: 0.821329	training's binary_logloss: 0.225846	valid_1's auc: 0.775437	valid_1's binary_logloss: 0.240852
[500]	training's auc: 0.831812	training's binary_logloss: 0.221851	valid_1's auc: 0.777047	valid_1's binary_logloss: 0.240281
[600]	training's auc: 0.8412	training's binary_logloss: 0.218269	valid_1's auc: 0.777948	valid_1's binary_logloss: 0.239961
[700]	training's auc: 0.849701	training's binary_logloss: 0.214961	valid_1's auc: 0.77841	valid_1's binary_logloss: 0.239814
[800]	trai

In [15]:
preds = clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1))[:, 1 ]
apps_all_test['TARGET'] = preds
apps_all_test[['SK_ID_CURR', 'TARGET']].to_csv('prev_baseline_tuning_02.csv', index=False)